In [1]:
import warnings
warnings.simplefilter("ignore")

import gc
from operator import length_hint
import numpy as np
from datetime import datetime,timedelta
from glob import glob

import pygrib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.basemap import shiftgrid

In [2]:
input_dir  = "../input/"
output_dir = "../output/"
flist_     = glob(input_dir+"h35*")
latcorners = [33,45]
loncorners = [23,48]
total_files=length_hint(flist_)

In [3]:
rgb = np.load("../auxilary/H35_cmap.npz")
newcmp = colors.ListedColormap(rgb["rgb"], "")

# for the multible input files

for file_ in flist_:
  date_=file_.split("/")[-1].split(".")[0].split("_")[1]
  grbs = pygrib.open(file_)
  grb = grbs.select()[0]
  del grbs
  data = grb.values
  lons = np.linspace(float(grb['longitudeOfFirstGridPointInDegrees'])-360 ,\
                     float(grb['longitudeOfLastGridPointInDegrees']), int(grb['Ni']))
  lats = np.linspace(float(grb['latitudeOfFirstGridPointInDegrees']), \
                     float(grb['latitudeOfLastGridPointInDegrees']), int(grb['Nj']))
  del grb
  lat0=np.where(lats<=latcorners[0])[0][0]
  lat1=np.where(lats<=latcorners[1])[0][0]
  lon0=np.where(lons>=loncorners[0])[0][0]
  lon1=np.where(lons>=loncorners[1])[0][0]
  trk_esc=data[lat1:lat0,lon0:lon1]
  del data
  trk_lons=lons[lon0:lon1]
  trk_lats=lats[lat1:lat0]
  del lons,lats
  trk_lon,trk_lat=np.meshgrid(trk_lons,trk_lats)
  del trk_lons,trk_lats
  m = Basemap(projection='merc', resolution='h', area_thresh=1000.,\
              llcrnrlat=latcorners[0],urcrnrlat=latcorners[1],\
              llcrnrlon=loncorners[0],urcrnrlon=loncorners[1])
  m.drawcountries()
  m.drawcoastlines()
  m1=m.pcolor(trk_lon,trk_lat,trk_esc,latlon=True,cmap=newcmp)
  plt.axis('off')
  plt.savefig(output_dir+"Turk_H35_"+date_+".png", transparent=True,\
            dpi=200, bbox_inches='tight', pad_inches = 0)
  plt.close()
  gc.collect()

In [4]:
file_=flist_[2]
date_=file_.split("/")[-1].split(".")[0].split("_")[1]
grbs = pygrib.open(file_)
grb = grbs.select()[0]
del grbs
data = grb.values
lons = np.linspace(float(grb['longitudeOfFirstGridPointInDegrees'])-360 ,\
                   float(grb['longitudeOfLastGridPointInDegrees']), int(grb['Ni']))
lats = np.linspace(float(grb['latitudeOfFirstGridPointInDegrees']), \
                   float(grb['latitudeOfLastGridPointInDegrees']), int(grb['Nj']))
del grb
lat0=np.where(lats<=latcorners[0])[0][0]
lat1=np.where(lats<=latcorners[1])[0][0]
lon0=np.where(lons>=loncorners[0])[0][0]
lon1=np.where(lons>=loncorners[1])[0][0]
trk_esc=data[lat1:lat0,lon0:lon1]
del data
trk_lons=lons[lon0:lon1]
trk_lats=lats[lat1:lat0]
del lons,lats
trk_lon,trk_lat=np.meshgrid(trk_lons,trk_lats)
del trk_lons,trk_lats
m = Basemap(projection='merc', resolution='h', area_thresh=1000.,\
            llcrnrlat=latcorners[0],urcrnrlat=latcorners[1],\
            llcrnrlon=loncorners[0],urcrnrlon=loncorners[1])
m.drawcountries()
m.drawcoastlines()
m1=m.pcolor(trk_lon,trk_lat,trk_esc,latlon=True,cmap=newcmp)
plt.axis('off')
plt.savefig(output_dir+"Turk_H35_"+date_+".png", transparent=True,\
          dpi=200, bbox_inches='tight', pad_inches = 0)
plt.close()
gc.collect()

0